# Jupyter notebook template for profilometry :dog: :bird:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from PyQt6 import QtWidgets   Uncomment this line if you want to use PyQt6
import plotly.io as pio

plt.rcParams["backend"] = 'QtAgg'
plt.rcParams["figure.figsize"] = (12, 7)
plt.rcParams["figure.dpi"] = 150
plt.rcParams["savefig.format"] = "svg"
plt.rcParams['animation.frame_format'] = "svg"
plt.rcParams['font.family'] = 'serif'

import sys
import os
wd = %pwd
%reload_ext autoreload
%autoreload 2
parent = os.path.dirname(wd)
fdel = os.path.sep

print(parent)
sys.path.append(parent)

import nanoscale_measurement as nano

/home/purpy-furcat/Documents/Code/Lab_Related


In [2]:
thickness = nano.profilometry.measure_thickness()



 proflometry module by James Bishop, use for analysis of profilometry trace data from DekTak profilomoter. Email: james.bishop@uts.edu.au if you have further questions. 


Remove outliers method,  Delta %:  10
Adding vertical line for measurement at x =  0.5806690265588661
Adding vertical line for measurement at x =  1.8738293227858769
Will fit substratself.all_datae baseline for leveling between, x1: 0.5806690265588661, x2: 1.8738293227858769
X1 and X2 set:  0.5806690265588661 1.8738293227858769
Fitting line to substrate between x1 and x2 using linear regression (np.polyfit(deg = 1)).
          Is a LBF with equal weighting to all points between x1 & x2.
Adding vertical line for measurement at x =  0.01774080019520552
Will fit substratself.all_datae baseline for leveling between, x1: 0.5806690265588661, x2: 1.8738293227858769


In [3]:
thickness_measure = thickness[0]
pro_trace = thickness[1]
pro_trace.data

In [7]:
pplot = nano.profilometry.plotly_plot(pro_trace)  # Plot all the data from profilometery using plotly for an interactive chart.
pplot.show()
# Display the figure in the browser
#pio.show(pplot, renderer='browser')

<img src="/home/purpy-furcat/Documents/Code/Lab_Related/Traces/salma_plot.svg" alt="Despiked, leveled, interpolated profilometry data." />

module_name:
    - __init__.py
    - main.py

In [ ]:
def qt_load_file_dialog(dialog_title: str = "Choose a file", initial_dir: str = wd,
                        file_types: str = "All Files (*);;Text Files (*.txt);;Excel Files (*.xlsx);;csv Files (*.csv)",):

    app = QtWidgets.QApplication.instance()  # Check if an instance already exists

    if not app:  # If not, create a new instance
        app = QtWidgets.QApplication(sys.argv)

    file_path, _ = QtWidgets.QFileDialog.getOpenFileName(None, dialog_title, initial_dir, file_types, options=QtWidgets.QFileDialog.Option.DontUseNativeDialog)

    return file_path

def profilometry_plot(data: pd.Series, **kwargs):
    fig, ax = plt.subplots(1, 1)
    ax.plot(data)
    ax.axhline(y=0, color='r', linestyle='--')
    ax.grid(visible=True, which='both', axis='both', ls = ":", lw = 0.75)
    # Increase the number of ticks on x and y axes
    ax.minorticks_on()
    ax.set_xlabel("x (mm)")
    ax.set_ylabel("Height (nm)")
    ax.set_title("Profilometry data")
    # ax.xaxis.set_major_locator(MaxNLocator(nbins=19))
    # ax.yaxis.set_major_locator(MaxNLocator(nbins=19))
    return fig, ax   


def find_closest_index(series: pd.Series, target_index_value: float) -> tuple:
    # Subtract the target value from the series and take the absolute value
    diff = abs((series.index - target_index_value))
    # Find the index of the minimum value in the resulting series
    closest_iloc = diff.argmin()
    # Find the iloc position of the closest index
    closest_index_value = series.index[closest_iloc]
    return closest_index_value, closest_iloc
